## Работа с Excel

Среди многообразия библиотек python для работы с Excel наиболее обширным функционалом для автоматизации из бесплатных решений на наш взгляд обладает библиотека **xlwings**.

Xlwings позволяет подключаться к открытым файлам Excel, читать и редактировать данные. Также после установки специальной надстройки для Excel, xlwings позволяет запускать скрипты python прямо из Excel и писать пользовательские функции для Excel на языке python.

### Начало работы

Импортируем библиотеку xlwings

In [1]:
import xlwings as xw

Для того, чтобы подключиться к открытой книге Excel, необходимо воспользоваться функцией ```xw.Book('File.xlsx')```, указав в качестве аргумента название файла. Если файл не открыт, но находится в рабочей директории, то аналогичная инструкция откроет этот файл. Функция ```Book()``` возвращает ссылку на объект типа ```Book```, с помощью которого можно читать и редактировать данные в соответствующей рабочей книге.

Откроем файл "Examples/Exchange rates/USD.xlsx" в Excel и попробуем подключиться к открытому документы из python.

In [2]:
wb = xw.Book('USD.xlsx') # Переменная wb теперь содержит объект Book, ссылающийся на открытую книгу USD.xlsx

Чтобы убедиться в том, что переменная wb теперь действительно содержит ссылку на наш файл Excel, попробуем прочитать какие-нибудь данные из файла. Детальное описание следующей инструкции будет приведено далее в тренинге.

In [3]:
print(wb.sheets[0].range('B1:C2').value)

[['data', 'curs'], [datetime.datetime(2021, 12, 31, 0, 0), 74.2926]]


Как видим, мы действительно получили содержимое диапазона B1:C2 в открытом файле: заголовок таблицы и первую строку с данными (дата и курс).

Теперь попробуем открыть файл Excel с помощью python. Для этого, как уже упоминалось, нужно воспользоваться аналогичной функцией и передать ей путь к файлу в качестве аргумента.

In [4]:
chf = xw.Book('Examples/Exchange rates/CHF.xlsx')

Альтернативно можно воспользоваться функцией ```xw.books.open('path')```

In [5]:
jpy = xw.books.open('Examples/Exchange rates/JPY.xlsx')

Сохранить и закрыть книгу можно при помощи функций ```save()``` и ```close()``` соответственно. Обратите внимание на то, что функция ```close()``` закрывает книгу без сохранения и, не выдавая стандартное окно с предложением сохранить изменения. Поэтому перед ее вызовом обязательно нужно убедиться, что все необходимые изменения сохранены - восстановить потерянные правки после закрытия будет не возможно!

In [6]:
jpy.save()
jpy.close()

### Объектная модель Xlwings

Объекты в xlwings имеют ту же иерархию (и те же названия), что и в Excel:
    
**Application -> Workbook -> Sheet -> Range**

Соответственно из любого объекта ```Book``` (как те, которые мы создавали в прошлых примерах), можно обратиться к дочерним объектам - листам. Такой объект называется ```Sheet```. В xlwings они сгруппированы в коллекцию ```sheets```, и к конкретным листам можно обращаться либо по их названию, либо по индексу.

In [7]:
print(wb.sheets[0].name) # Вывести имя первого листа (индексация с нуля!)

RC


In [8]:
print(wb.sheets['RC'].name) # Аналогичный результат, но уже с обращением к листу по имени, а не по индексу

RC


Для создания нового листа необходимо воспользоваться функцией ```add()```. Опционально можно передать ей в качестве аргументов имя нового листа и его расположение (перед или после какого-либо из существующих листов).

In [9]:
wb.sheets.add() # Добавлен новый лист с именем по умолчанию (н.п. "Sheet1")

<Sheet [USD.xlsx]Sheet1>

In [10]:
wb.sheets.add('new') # Добавлен новый лист с именем "new"

<Sheet [USD.xlsx]new>

In [11]:
wb.sheets.add('before', before='RC') # Добавлен новый лист с именем "before" перед листом "RC"

<Sheet [USD.xlsx]before>

In [12]:
wb.sheets.add('after', after='RC') # Добавлен новый лист с именем "after" после листа "RC"

<Sheet [USD.xlsx]after>

Дочерним объектом для листа (```Sheet```) является объект диапазон - ```Range```. К диапазону можно обращаться аналогично тому, как это осуществляется в Excel - по адресу (н.п. 'B1' или 'A1:B2') или по имени (другие способы останутся за пределами данного тренинга, но они описаны в документации к xlwings).

Давайте попробуем прочитать значение ячейки A1 в файле "USD.xlsx" при помощи свойства ```value``` листа.

In [13]:
print(wb.sheets['RC'].range('A1').value)

nominal


Теперь давайте вернемся в Excel и назовем диапазон A1:D1 на листе RC *title*. А затем попробуем прочитать данные из него в python по имени этого диапазона.

In [14]:
print(wb.sheets['RC'].range('title').value)

['nominal', 'data', 'curs', 'cdx']


### Работа с диапазонами

Как мы уже видели выше, данные из диапазона в Excel можно получать при помощи свойства ```value``` листа. Однако аналогично можно и записывать значения в ячейку, присвоив данному свойству нужное значение.

In [15]:
wb.sheets['RC'].range('F1').value = '3d average'

In [16]:
rates = wb.sheets['RC'].range('C2:C4').value # Получаем массив значений курса за первые 3 дня
average = sum(rates) / len(rates) # Находим среднее из этих трех значений
wb.sheets['RC'].range('F2').value = average # Записываем получившийся результат в ячейку F2

Аналогично можно записать в ячейку формулу, которая будет динамически работать в Excel. Например, создадим в ячейке F3 формулу, дающую аналогичный результат предыдущим вычислениям.

In [17]:
wb.sheets['RC'].range('F3').value = '=AVERAGE(C2:C4)'

Как мы уже видели в примере с чтением заголовка таблицы по имени диапазона, данные из диапазонов в Excel возвращаются в python в виде массива. В случае с двумерным диапазоном массив также будет двумерным, где первым уровнем будут строки таблицы, а вторым - столбцы.

```
[
        [Столбец 1, Столбец 2],   -> строка 1
        [Столбец 1, Столбец 2]    -> строка 2
]
```

Для примера получим данные курса за первые два дня вместе с датами.

In [18]:
print(wb.sheets['RC'].range('B1:C3').value)

[['data', 'curs'], [datetime.datetime(2021, 12, 31, 0, 0), 74.2926], [datetime.datetime(2021, 12, 30, 0, 0), 73.6514]]


Массив содержит три элемента (также массивы), которые представляют из себя три строки в Excel: <br>
(1) заголовок *(['data', 'curs'])*, <br>
(2) дата и курс в первый день *([datetime.datetime(2021, 12, 31, 0, 0), 74.2926])* и <br>
(3) дата и курс во второй день *([datetime.datetime(2021, 12, 30, 0, 0), 73.6514])*

Каждый из этих элементов также является массивом и содержит по два элемента, которые представляют из себя столбцы - дата и курс.

Запись диапазонов в Excel осуществляется по тому же принципу. Для записи строк достаточно перечислить значения каждого столбца в строке в одномерном массиве.

In [19]:
wb.sheets['RC'].range('F5').value = [1, 2, 3]

Для записи столбца из нескольких строк необходимо использовать вложенные массивы (двумерный массив), так как каждая строка должна представляться отдельным массивом.

In [20]:
wb.sheets['RC'].range('F5').value = [[1], [2], [3]]

Более удобным способом получения аналогичного результата является испольозвание опции ```transpose```.

In [21]:
arr = [10, 20, 30]
wb.sheets['RC'].range('F5').options(transpose=True).value = arr

Соответственно двумерные диапазоны записываются при помощи двумерных массивов, где внешний уровень - это строки, а вложенный - столбцы.

In [22]:
wb.sheets['RC'].range('F5').value = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

Для получения полного диапазона, начиная от определенной ячейки, используется функция ```expand()```. <br>
Она используется с тремя основными параметрами:<br>
1. ```down``` - выделяет диапазон до конца вниз, аналогично использованию комбинации Ctrl+Shift+↓ в Excel
2. ```right``` - выделяет диапазон до конца вправо, аналогично использованию комбинации Ctrl+Shift+→ в Excel
3. ```table``` - объединяет две предыдущие команды - выделяет диапазон целиком вниз и вправо

In [23]:
print(wb.sheets['RC'].range('A1').expand('down').address)

$A$1:$A$249


In [24]:
print(wb.sheets['RC'].range('A1').expand('right').address)

$A$1:$D$1


In [25]:
print(wb.sheets['RC'].range('A1').expand('table').address)

$A$1:$D$249


### Выполнение кода на Python из Excel

Инструмент xlwings также включает специальную надстройку для Excel со следующим функционалом:
 - запуск скриптов, написанных на Python из Excel
 - написание пользовательских функций на языке Python
 
Для установки надстройки закройте Excel и выполните в командной строке следующую инструкцию:

 ```xlwings addin install```
 
Далее в Excel необходимо добавить xlwings в References для того, чтобы ее могли использовать макросы в Excel. Для этого нужно на вкладке Developer нажать кнопку Visual Basic. В меню Tools выбрать пункт References, в появившемся окне найти пункт xlwings и убедиться, что он отмечен галочкой.

Для запуска скрипта на Python в VBA (Visual Basic for Applications - язык программирования, используемый в Excel) используется специальная функция ```RunPython```. В качестве параметра ей передается код на языке Python. Для удобства редактирования кода, лучше вынести его в отдельный модуль и импортировать в Excel.

Например, создадим новый файл "script.py" и сохраним в той же папке новый пустой файл Excel.
В файле "script.py" напишем следующий скрипт для получения котировок акции по ее тикеру за последний год.

In [ ]:
def getQuotes():
    
    import xlwings as xw
    wb = xw.Book.caller() # Ссылка на книгу Excel, из которой был вызван скрипт
    ticker = wb.sheets[0].range('ticker').value # Получение тикера из ячейки с именем "ticker" в Excel

    # Код необходимый для корректной работы импорта котировок (не будет разбираться в рамках данного тренинга)
    import requests
    from urllib3.exceptions import InsecureRequestWarning
    session = requests.Session()
    session.verify = False
    requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

    import yfinance as yf # yfinance - популярная библиотека для получения данных (в т.ч. котировок) из Yahoo Finance
    quotes = yf.Ticker(ticker, session=session).history(period='1y') # Импорт котировок за последний год
    wb.sheets[0].range('output').value = quotes # Вывод результата в Excel

Теперь вернемся в Excel и зададим имена ячеек для ввода и вывода данных.

Для получения имени тикера мы использовали в нашем скрипте ячейку с названием 'ticker'. Поэтому зададим соответствующее имя, например, ячейке A1 в Excel. Введем туда какой-нибудь популярный тикер вроде AAPL или MSFT.

Для вывода будем использовать к примеру диапазон, начиная с ячейки C1. Для этого присвоим ячейке C1 имя 'output'.

Теперь создадим новый VBA модуль. Для этого необходимо открыть Visual Basic на вкладке Developer и в меню Insert выбрать Module. Напишем простую функцию, импортирующую наш скрипт из файла и запускающую функцию getQuotes в этом скрипте.

In [ ]:
Sub runPythonScript()
  RunPython "import script; script.getQuotes()"
End Sub

Запустим нашу функцию в VBA, нажав F5 в редакторе. Если все было выполнено корректно, VBA должен импортировать скрипт из файла 'script.py' и выполнить в нем функцию ```getQuotes()```. Скрипт прочитает из Excel наименование тикера, получит котировки за последний год и вернет их в Excel в диапазон, начиная с ячейки с именем 'output'.